In [1]:
!pip install pyomo


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install openpyxl



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# Install the needed libraries
import pandas as pd
import numpy as np
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
from pyomo.environ import *

In [4]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [5]:
# Import the data
data = pd.read_excel('data_python_modified.xlsx', sheet_name='data', index_col=0)
real = pd.read_excel('data_python_modified.xlsx', sheet_name='реал', index_col=0)
bandwidth = pd.read_excel('data_python_modified.xlsx', sheet_name='ПС', index_col=0)
match = pd.read_excel('data_python_modified.xlsx', sheet_name='match', index_col=0)

In [6]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф ж/д" : "t1",
        "Тариф хранение" : "t2",
        "Тариф бренд" : "t3",
        "Тариф ВЛ" : "t4",
        "Плечо, км" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НГ" : "brand",
        "НГ_ПС" : "product",
        "Объем" : "volume"} 

In [7]:
# Rename the names in dataframes
data.rename(columns=names, inplace=True)
real.rename(columns=names, inplace=True)
bandwidth.rename(columns=names, inplace=True)
match.rename(columns=names, inplace=True)

In [8]:
# Function for calculating the tarif
def calculate_tarif(row):
    if row["distance"] <= 50:
        return row["t1"] + row["t2"] + row["t3"] + row["t4"]
    else:
        return row["t1"] + row["t2"] + row["t3"] +  row["t4"] * row["distance"]

In [9]:
# Calculate cost
data['cost'] = data.apply(lambda row: calculate_tarif(row), axis=1)

In [10]:
# Checking dates
dates = pd.DataFrame({"date" : data['date'].unique()})
dates

,date
0,2023-04-01
1,2023-03-01


In [11]:
# Save the first date to the t 
t = dates.iloc[0, 0]

In [12]:
# Subset for the time period
data = data[data['date'] == t]
real = real[real['date'] == t]
bandwidth = bandwidth[bandwidth['date'] == t]
    
# Adjust datatypes
data['origin'] = data['origin'].map(lambda x: int(x.strip('Нефтебаза ')))
data['point'] = data['point'].map(lambda x: int(x.strip('АЗС ')))
real['point'] = real['point'].map(lambda x: int(x.strip('АЗС ')))
bandwidth['origin'] = bandwidth['origin'].map(lambda x: int(x.strip('Нефтебаза ')))

In [13]:
# Create some dictionaries for later usage
match_brand_product = dict(zip(match['brand'], match['product']))
match_brand_number = dict(enumerate(match['brand']))
match_product_number = dict(enumerate(match['product']))

match_product_brand = dict()
for key, value in match_brand_product.items():
    match_product_brand.setdefault(value, list()).append(key)
    
match_number_brand = dict()
for key, value in match_brand_number.items():
    match_number_brand[value] = key
    
match_number_product = dict()
for key, value in match_product_number.items():
    match_number_product[value] = key

In [14]:
# Store origin with infinite supply in a separate list
origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())

In [15]:
# Function for calculating the tarif
def create_var_Name(row):
    return 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))

In [16]:
# Reorganize the data
columns_to_keep = ['origin', 'point', 'brand', 'product', 'cost']

df = data[columns_to_keep].reset_index().drop('index', axis=1).reset_index()

# Code brand, product
df['brand_number'] = df['brand'].map(lambda x: match_number_brand.get(x))
df['product_number'] = df['product'].map(lambda x: match_number_product.get(x))

# Create a name variable
df['var_Name'] = df.apply(lambda row: create_var_Name(row), axis=1) 

In [17]:
# Create demand df
demand = real[['point', 'brand', 'product', 'volume']]
demand.rename(columns={'volume':'demand'}, inplace=True)
demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))

In [18]:
# Create supply df
supply = bandwidth[['origin', 'product', 'volume']]
supply.rename(columns={'volume':'supply'}, inplace=True)
supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))

In [19]:
df

,index,origin,point,brand,product,cost,brand_number,product_number,var_Name
0,0,0,0,Бензин 100 бренд,АИ 100,3268.488947,0,0,P0O0_brand_0
1,1,0,0,Бензин 92,Аи 92,3268.488947,1,1,P0O0_brand_1
2,2,0,0,Бензин 95,Аи 95,3268.488947,2,3,P0O0_brand_2
3,3,0,0,Бензин 95 бренд,Аи 95,3338.651597,3,3,P0O0_brand_3
4,4,0,0,Топливо дизельное с присадками летнее,ДТ,3133.526439,4,4,P0O0_brand_4
...,...,...,...,...,...,...,...,...,...
355,355,1,47,Топливо дизельное с присадками летнее,ДТ,4734.995826,4,4,P47O1_brand_4
356,356,1,48,Бензин 92,Аи 92,4808.465100,1,1,P48O1_brand_1
357,357,1,48,Бензин 95,Аи 95,4808.465100,2,3,P48O1_brand_2
358,358,1,48,Бензин 95 бренд,Аи 95,4854.005100,3,3,P48O1_brand_3


In [20]:
#conda install -c conda-forge pyomo coincbc

# Pyomo

In [21]:
model = ConcreteModel()

In [22]:
# задаем переменные, в данном случае они все непрерывные
model.x = Var(range(len(df)), domain=Reals, bounds=(0.0,None))
#within=NonNegativeReals

In [23]:
c = np.array(df['cost'])

In [24]:
# объявление целевой функции и передача в модель
obj_expr = sum(c[i] * model.x[i] for i in model.x)


In [25]:
obj_expr 

#### Define restriction #1: demand equals const

In [26]:
model.constraints = ConstraintList()

In [27]:
n_constraint = 1
brands = df['brand_number'].unique()

In [28]:
for b in brands:
    df1 = df[df['brand_number'] == b]
    points = df1['point'].unique()
    
    for p in points:
        df2 = df1[df1['point'] == p]
        const = float(demand[(demand['point'] == p) & (demand['brand_number'] == b)]['demand'])
        # Add a restriction on demand == const
        # problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) == const, str(n_constraint)
        #constraints.append(cp.sum(x[df2.index]) == const)
        #expr = sum(model.x[df2.index])
        expr = sum(model.x[i] for i in df2.index)
        model.constraints.add(expr == const)
        n_constraint +=1

In [29]:
#model.constraints.add( expr >= 1 )

#### Define restriction #2: supply less or equal than const;  restriction #4: supply greater or equal to const * percent_loading

In [30]:
# Define percents of loading demanded by management
percents_loading = np.arange(0.5, 1, 0.1)
percent_loading = percents_loading[0]

In [31]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [32]:
# Set for storing already processed brands 
brands_accounted_for = set()

In [33]:
for b in brands:
    
    if b in brands_accounted_for:
        continue
    else:
        brands_overlap_numbers = merge_supply(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)
        
        df1 = df[df['brand_number'].isin(brands_overlap_numbers)]
        origins = df1['origin'].unique()
        
        for o in origins:
            
            if o in origins_with_inf_supply:
                continue
            else:
                
                df2 = df1[df1['origin'] == o]
                product_name = match_brand_product[match_brand_number[b]]
                const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
                
                # Add a restriction on supply <= upper_const
                model.constraints.add(sum(model.x[i] for i in df2.index) <= const)
                #constraints.append(cp.sum(x[df2.index]) <= const)
                n_constraint +=1

                # Add a restriction on supply >= lower_const
                const *= percent_loading
                #constraints.append(cp.sum(x[df2.index]) >= const)
                model.constraints.add(sum(model.x[i] for i in df2.index) >= const)
                n_constraint +=1        

#### Define restriction #3: all brands for one point should come from the same origin

In [34]:
M = 10**9

In [35]:
# Function for creating dummies names
def create_dummy_Name(row):
    return 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), int(row["point"]), int(row["origin"])

In [36]:
# Create a name variable
[*o] = df.apply(lambda row: create_dummy_Name(row), axis=1)

In [37]:
df_dummies = pd.DataFrame(o, columns=['dummy_Name', 'point', 'origin'])

In [38]:
df_dummies = df_dummies.reset_index()

In [39]:
df_dummies

,index,dummy_Name,point,origin
0,0,y_P0O0,0,0
1,1,y_P0O0,0,0
2,2,y_P0O0,0,0
3,3,y_P0O0,0,0
4,4,y_P0O0,0,0
...,...,...,...,...
355,355,y_P47O1,47,1
356,356,y_P48O1,48,1
357,357,y_P48O1,48,1
358,358,y_P48O1,48,1


In [40]:
df_dummies = df_dummies.drop_duplicates(subset=['dummy_Name', 'point', 'origin'])

In [41]:
df_dummies

,index,dummy_Name,point,origin
0,0,y_P0O0,0,0
5,5,y_P1O0,1,0
10,10,y_P2O0,2,0
14,14,y_P3O0,3,0
18,18,y_P4O0,4,0
...,...,...,...,...
341,341,y_P44O1,44,1
345,345,y_P45O1,45,1
349,349,y_P46O1,46,1
352,352,y_P47O1,47,1


In [42]:
#### Creating our cvxpy variable of assignments
#dummies = cp.Variable(len(df_dummies), boolean=True)
model.dummies = Var(range(len(df_dummies)), domain=Integers, bounds=(0.0, 1.0))

In [43]:
model.dummies

In [44]:
df_dummies.reset_index(inplace=True)

In [45]:
df_dummies.drop(columns=['index', 'level_0'], inplace=True)

In [46]:
df_dummies

,dummy_Name,point,origin
0,y_P0O0,0,0
1,y_P1O0,1,0
2,y_P2O0,2,0
3,y_P3O0,3,0
4,y_P4O0,4,0
...,...,...,...
93,y_P44O1,44,1
94,y_P45O1,45,1
95,y_P46O1,46,1
96,y_P47O1,47,1


In [47]:
points = df['point'].unique()
origins = df['origin'].unique()

for p in points:
    for o in origins:
        df1 = df[(df['point'] == p) & (df['origin'] == o)]
        
        
        df_d = df_dummies[(df_dummies['point'] == p) & (df_dummies['origin'] == o)]
        
        #constraints.append(cp.sum(x[df1.index]) <= M * dummies[df_d.index])
        for j in df_d.index:
            m = model.dummies[j] 
            model.constraints.add(sum(model.x[i] for i in df1.index) <= M  * m )
        n_constraint += 1

In [48]:
n_constraint

287

In [49]:
const = 1

In [50]:
for p in points:
    
    df_d = df_dummies[df_dummies['point'] == p]

    #constraints.append(cp.sum(dummies[df_d.index]) == const)
    model.constraints.add(sum(model.dummies[i] for i in df_d.index) == const)
    n_constraint += 1

In [51]:
model.objective = Objective(expr=obj_expr, sense=minimize)

In [52]:
#!pyomo help --solvers

In [53]:
# solve with cbc
start = time.time()
opt = SolverFactory('cbc')
res = opt.solve(model)
x_opt = [round(model.x[i].value, 3) for i in model.x]
print(f"Решение cbc: {list(x_opt)} {res.solver.termination_condition}")
print(f"Выручка: {round(model.objective(), 0)}")
stop = time.time()
print ("Время :")
print(stop - start)

Решение cbc: [0.0, 0.0, 0.0, 0.0, 0.0, 23.718, 240.303, 231.371, 49.888, 54.458, 0.0, 0.0, 0.0, 0.0, 69.617, 156.741, 246.201, 78.749, 17.18, 199.029, 154.649, 39.69, 204.282, 0.0, 0.0, 0.0, 0.0, 33.145, 286.426, 277.954, 53.752, 19.221, 208.336, 174.274, 48.919, 139.34, 0.0, 0.0, 0.0, 0.0, 42.385, 268.45, 276.059, 67.783, 108.141, 18.717, 182.128, 175.193, 40.864, 110.715, 29.952, 241.909, 246.868, 54.434, 46.747, 150.437, 68.077, 38.306, 190.244, 98.428, 8.061, 144.6, 92.44, 19.916, 215.386, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.213, 104.475, 90.55, 25.909, 658.264, 4.738, 185.14, 108.562, 19.583, 52.247, 15.237, 155.832, 174.709, 47.087, 366.786, 7.962, 147.817, 116.408, 27.221, 393.64, 14.28, 113.705, 116.141, 32.012, 488.357, 0.0, 0.0, 134.086, 105.576, 154.898, 0.0, 0.0, 0.0, 8.316, 78.056, 72.369, 20.35, 510.134, 165.674, 106.905, 247.038, 10.245, 42.924, 0.0, 26.563, 43.263, 0.0, 105.739, 62.543, 124.484, 106.664, 49.957, 73.167, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 190.

In [54]:
y_opt = [round(model.dummies[i].value, 3) for i in model.dummies]
print(f"Решение cbc: {list(y_opt)} {res.solver.termination_condition}")

Решение cbc: [0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0] optimal
